In [1]:
import argparse
import configparser
import time
import warnings
from distutils.util import strtobool
from json import loads

import numpy as np
import openmm as mm
from openmm import unit

from parmed.exceptions import OpenMMWarning

import cosmo

In [2]:
config_file = 'md.ini'

In [4]:
# Default values:
# Set default values
md_steps = 1000
dt = 0.01
nstxout = 10
nstlog = 10
nstcomm = 100
model = 'hps_urry'
tcoupl = True
ref_t = 300.0
pcoupl = False
ref_p = 1.0
frequency_p = 25
pbc = False
device = 'CPU'
ppn = 1
restart = False
minimize = True

# Other attributes
tau_t = None
box_dimension = None
protein_code = None
checkpoint = None
pdb_file = None

In [5]:
# Read config file
print(f"Reading simulation parameters from {config_file} file...")
print('-' * 70)
print('Simulation parameters:')
config = configparser.ConfigParser()
config = configparser.ConfigParser(inline_comment_prefixes=("#", ";"))
config.read(config_file)
params = config['OPTIONS']

# Update simulation parameters
md_steps = int(params.get('md_steps', md_steps))
print(f'  md_steps: {md_steps}')
dt = float(params.get('dt', dt)) * unit.picoseconds
print(f'  dt: {dt}')
nstxout = int(params['nstxout'])
print(f'  nstxout: {nstxout}')
nstlog = int(params['nstlog'])
print(f'  nstlog: {nstlog}')
nstcomm = int(params.get('nstcomm', nstcomm))
print(f'  nstcomm: {nstcomm}')
model = params.get('model', model)
print(f'  model: {model}')
tcoupl = bool(strtobool(params.get('tcoupl', tcoupl)))
if tcoupl:
    ref_t = float(params['ref_t']) * unit.kelvin
    tau_t = float(params['tau_t']) / unit.picoseconds
    print(f'  tcoupl: on (ref_t={ref_t}, tau_t={tau_t})')
else:
    print('  tcoupl: off')
pbc = bool(strtobool(params.get('pbc', pbc)))
if pbc:
    box_dimension = loads(params['box_dimension'])
    print(f'  pbc: on (box_dimension={box_dimension} nm)')
else:
    box_dimension = None
    print('  pbc: off')
pcoupl = bool(strtobool(params.get('pcoupl', pcoupl)))
if pcoupl:
    assert pbc, "Pressure coupling requires box dimensions and periodic boundary condition is on"
    ref_p = float(params['ref_p']) * unit.bar
    frequency_p = int(params['frequency_p'])
    print(f'  pcoupl: on (ref_p={ref_p}, frequency={frequency_p})')
else:
    print('  pcoupl: off')
pdb_file = params['pdb_file']
print(f'  pdb_file: {pdb_file}')
protein_code = params['protein_code']
print(f'  output_prefix: {protein_code}')
checkpoint = params.get('checkpoint', checkpoint)
device = params.get('device', device)
print(f'  device: {device}')
if device == "CPU":
    ppn = int(params.get('ppn', ppn))
    print(f'  threads: {ppn}')
restart = bool(strtobool(params.get('restart', restart)))
print(f'  restart: {restart}')
if restart:
    minimize = False
else:
    minimize = bool(strtobool(params.get('minimize', minimize)))
    print(f'  minimize: {minimize}')
print('-' * 70)

Reading simulation parameters from md.ini file...
----------------------------------------------------------------------
Simulation parameters:
  md_steps: 10000
  dt: 0.01 ps
  nstxout: 100
  nstlog: 100
  nstcomm: 100
  model: synthesis_kr
  tcoupl: on (ref_t=310.0 K, tau_t=0.01 /ps)
  pbc: off
  pcoupl: off
  pdb_file: complex.pdb
  output_prefix: complex
  device: GPU
  restart: False
  minimize: False
----------------------------------------------------------------------


In [21]:
hps_model = cosmo.models.buildHPSModel(pdb_file, minimize=minimize, model=model, box_dimension=box_dimension)

Generating CA/P coarse-grained model for structure from file complex.pdb

Checking input structure file ...
Be sure that you do not have missing residues in the initial structure. At the moment, I will not take care of that
Setting up geometrical parameters ...
----------------------------------------------------------------------
Keeping only alpha carbon and phosphate P atoms in topology
There are 18 chain(s) in the input file.
Added 1987 atoms (860 CA, 1127 P)
Added 1969 bonds
Setting CA/P masses to their average residue mass.
Setting CA/P charge to their residue charge.
Setting CA/P atom radii to their statistical residue radius.
Setting CA/P hydropathy scale to their residue, Using synthesis_kr scale.
Adding default bond force constant: bond_force_constant: 8368.0 kj/mol/nm^2

----------------------------------------------------------------------
Adding Forces:
Added Harmonic Bond Forces
------------------------------
Setting Coulomb interaction ...
Set cutoff NonPeriodic ...
Use 

In [16]:
for a in hps_model.topology.atoms():
    print(a.index, a.residue.chain.index, a.residue.chain.id)

0 0 A
1 1 B
2 1 B
3 2 C
4 2 C
5 2 C
6 2 C
7 2 C
8 2 C
9 2 C
10 2 C
11 2 C
12 2 C
13 2 C
14 2 C
15 2 C
16 2 C
17 2 C
18 2 C
19 2 C
20 2 C
21 2 C
22 2 C
23 2 C
24 2 C
25 2 C
26 2 C
27 2 C
28 2 C
29 2 C
30 2 C
31 2 C
32 2 C
33 2 C
34 2 C
35 2 C
36 2 C
37 2 C
38 2 C
39 2 C
40 2 C
41 2 C
42 2 C
43 2 C
44 2 C
45 2 C
46 2 C
47 2 C
48 2 C
49 2 C
50 2 C
51 2 C
52 2 C
53 2 C
54 2 C
55 2 C
56 2 C
57 2 C
58 2 C
59 2 C
60 2 C
61 2 C
62 2 C
63 2 C
64 2 C
65 2 C
66 2 C
67 2 C
68 2 C
69 2 C
70 2 C
71 2 C
72 2 C
73 2 C
74 2 C
75 2 C
76 2 C
77 2 C
78 2 C
79 2 C
80 2 C
81 2 C
82 2 C
83 2 C
84 2 C
85 2 C
86 2 C
87 2 C
88 2 C
89 2 C
90 2 C
91 2 C
92 2 C
93 2 C
94 2 C
95 2 C
96 2 C
97 2 C
98 2 C
99 2 C
100 2 C
101 2 C
102 2 C
103 2 C
104 2 C
105 2 C
106 2 C
107 2 C
108 2 C
109 2 C
110 2 C
111 2 C
112 2 C
113 2 C
114 2 C
115 2 C
116 2 C
117 2 C
118 2 C
119 2 C
120 2 C
121 2 C
122 2 C
123 2 C
124 2 C
125 2 C
126 2 C
127 2 C
128 2 C
129 2 C
130 2 C
131 2 C
132 2 C
133 2 C
134 2 C
135 2 C
136 2 C
137 2 C
138 2 

In [20]:
for c in hps_model.topology.chains():
    print(c.id)

A
B
C
E
F
G
L
N
P
S
T
U
V
W
A
C
E
8


In [22]:
hps_model.particles_mass

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0